In [1]:
import pandas as pd
import networkx as nx
import os
import zipfile
import matplotlib as plt

In [2]:
# modify display options to make sure we can see full texts fields
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
# unzip all the files in the folder, but first check if the zip files are there
# check if the folder exist
for i in os.listdir('../data/capital_bikes/'):
    if i.endswith('csv'):
        pass
    elif i.endswith('zip'):
        with zipfile.ZipFile('../data/capital_bikes/'+i) as item: # treat the file as a zip
            item.extractall('../data/capital_bikes/')

In [3]:
# read all the files
list_dataframes = []

for i in os.listdir('../data/capital_bikes/'):
    if i.endswith('csv'):
        a = pd.read_csv('../data/capital_bikes/'+i)
        list_dataframes.append(a)

In [4]:
# check that dataframes were parsed alright
list_dataframes[0]

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,257,2015-07-01 00:00:25,2015-07-01 00:04:43,31116,California St & Florida Ave NW,31117,15th & Euclid St NW,W21516,Member
1,2683,2015-07-01 00:00:26,2015-07-01 00:45:10,31247,Jefferson Dr & 14th St SW,31247,Jefferson Dr & 14th St SW,W00476,Casual
2,2349,2015-07-01 00:02:39,2015-07-01 00:41:49,31247,Jefferson Dr & 14th St SW,31247,Jefferson Dr & 14th St SW,W20384,Casual
3,826,2015-07-01 00:02:49,2015-07-01 00:16:36,31229,New Hampshire Ave & T St NW,31602,Park Rd & Holmead Pl NW,W00972,Member
4,1192,2015-07-01 00:02:52,2015-07-01 00:22:45,31245,7th & R St NW / Shaw Library,31630,15th & East Capitol St NE,W21272,Member
...,...,...,...,...,...,...,...,...,...
1056361,1538,2015-09-30 23:54:39,2015-10-01 00:20:17,31258,Lincoln Memorial,31277,17th & G St NW,W21241,Casual
1056362,1474,2015-09-30 23:54:53,2015-10-01 00:19:27,31258,Lincoln Memorial,31277,17th & G St NW,W21239,Casual
1056363,705,2015-09-30 23:56:33,2015-10-01 00:08:19,31201,15th & P St NW,31275,New Hampshire Ave & 24th St NW,W21283,Member
1056364,1068,2015-09-30 23:56:47,2015-10-01 00:14:36,31101,14th & V St NW,31615,6th & H St NE,W21169,Member


In [5]:
# concatenate the list of dataframes
bike_trips_data = pd.concat(list_dataframes)
bike_trips_data.head(2)
del list_dataframes

In [ ]:
# we can see there two different data set schemas and as a result they pasted next to each other

In [7]:
# check the number of observations in the data
print(len(bike_trips_data))

27861668


In [8]:
# the difference in schema is due to a change on Capitol Bike Share collected bike trip data so 
# to analyze the data I need to subset data for the years 2010 to 2019 and another subset for 2020
bike_trips_data_2019 = bike_trips_data[pd.DatetimeIndex(bike_trips_data['Start date']).year<=2019]
bike_trips_data_2019.head(5)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source,target,weight,is_equity,OBJECTID,ID,ADDRESS,TERMINAL_NUMBER,LATITUDE,LONGITUDE,INSTALLED,LOCKED,INSTALL_DATE,REMOVAL_DATE,TEMPORARY_INSTALL,NUMBER_OF_BIKES,NUMBER_OF_EMPTY_DOCKS,X,Y,SE_ANNO_CAD_DATA,OWNER
0,257.0,2015-07-01 00:00:25,2015-07-01 00:04:43,31116.0,California St & Florida Ave NW,31117.0,15th & Euclid St NW,W21516,Member,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2683.0,2015-07-01 00:00:26,2015-07-01 00:45:10,31247.0,Jefferson Dr & 14th St SW,31247.0,Jefferson Dr & 14th St SW,W00476,Casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2349.0,2015-07-01 00:02:39,2015-07-01 00:41:49,31247.0,Jefferson Dr & 14th St SW,31247.0,Jefferson Dr & 14th St SW,W20384,Casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,826.0,2015-07-01 00:02:49,2015-07-01 00:16:36,31229.0,New Hampshire Ave & T St NW,31602.0,Park Rd & Holmead Pl NW,W00972,Member,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1192.0,2015-07-01 00:02:52,2015-07-01 00:22:45,31245.0,7th & R St NW / Shaw Library,31630.0,15th & East Capitol St NE,W21272,Member,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# we create the same subset for 2020
bike_trips_data_2020_part1 = bike_trips_data[pd.DatetimeIndex(bike_trips_data['Start date']).year>=2020]
bike_trips_data_2020_part2 = bike_trips_data[pd.DatetimeIndex(bike_trips_data['started_at']).year>=2020]

In [11]:
# we subset to get the data keeping the station number of origin and destination. We also add a weight of 1 to 
# the data so we add up the total number of trips across the nodes and use that as a proxy of edge weight
print(len(bike_trips_data_2019))
bike_trips_data_graph_2019 = bike_trips_data_2019[['Start station','End station']].copy()
bike_trips_data_graph_2019['trips'] = 1
bike_trips_data_graph_2019 = bike_trips_data_graph_2019.groupby(['Start station',
                                                       'End station']).sum().reset_index()
print(len(bike_trips_data_graph_2019))

25702099
133263


In [15]:
# now we repeat for 2020
# data for 2020 registers when there was a change in the data fields used to record information. So I need
# to account for the change in how they capture information. To do this I subset 
bike_trips_data_2020_part1 = bike_trips_data_2020_part1[['Start station', 'End station']]
bike_trips_data_2020_part1 = bike_trips_data_2020_part1.rename(columns = {'Start station':'source', 
                                                                          'End station':'target'})
bike_trips_data_2020_part2 = bike_trips_data_2020_part2[['start_station_name','end_station_name']]
bike_trips_data_2020_part2 = bike_trips_data_2020_part2.rename(columns = {'start_station_name':'source', 
                                                                          'end_station_name':'target'})

bike_trips_data_2020 = pd.concat([bike_trips_data_2020_part1, bike_trips_data_2020_part2])
print(len(bike_trips_data_2020))
bike_trips_data_2020['weight'] = 1
bike_trips_data_2020 = bike_trips_data_2020.groupby(['source',
                                                     'target']).sum().reset_index()
print(len(bike_trips_data_2020))
bike_trips_data_2020.head()

1935143
91853


,source,target,weight
0,10th & E St NW,10th & E St NW,272
1,10th & E St NW,10th & Florida Ave NW,8
2,10th & E St NW,10th & G St NW,13
3,10th & E St NW,10th & K St NW,27
4,10th & E St NW,10th & Monroe St NE,16


In [16]:
#bike_trips_data_sub.columns
bike_trips_data_2020.head()

,source,target,weight
0,10th & E St NW,10th & E St NW,272
1,10th & E St NW,10th & Florida Ave NW,8
2,10th & E St NW,10th & G St NW,13
3,10th & E St NW,10th & K St NW,27
4,10th & E St NW,10th & Monroe St NE,16


In [17]:
# The network will have a few trips where the origin and destinatio are the same, we will filter those observations
# from the dataframe
print(len(bike_trips_data_graph_2019))
bike_trips_data_graph_2019 = bike_trips_data_graph_2019[bike_trips_data_graph_2019['Start station']!=
                                              bike_trips_data_graph_2019['End station']]
bike_trips_data_graph_2019 = bike_trips_data_graph_2019.reset_index()
bike_trips_data_graph_2019 = bike_trips_data_graph_2019.drop(['index'], axis=1)
print(len(bike_trips_data_graph_2019))
bike_trips_data_graph_2019.head()

133263
132598


,Start station,End station,trips
0,10th & E St NW,10th & Florida Ave NW,100
1,10th & E St NW,10th & G St NW,419
2,10th & E St NW,10th & K St NW,1165
3,10th & E St NW,10th & Monroe St NE,63
4,10th & E St NW,10th & U St NW,319


In [18]:
# repeat for 2020
bike_trips_data_graph_2020 = bike_trips_data_2020[bike_trips_data_2020['source']!=
                                              bike_trips_data_2020['target']]
bike_trips_data_graph_2020 = bike_trips_data_graph_2020.reset_index()
bike_trips_data_graph_2020 = bike_trips_data_graph_2020.drop(['index'], axis=1)

In [19]:
#bike_trips_data_2019[bike_trips_data_2019['Start station number']==0.0]
start_ = bike_trips_data_2019['Start station'].tolist()
end_ = bike_trips_data_2019['End station'].tolist()
list_stations = set(start_+end_)
del start_, end_
len(list_stations)

667

In [20]:
# now we to also process a mapping file of the location of the files to get the latitute and longitude of the 
# stations. That information is available here: 
bike_locations = pd.read_csv('https://opendata.arcgis.com/datasets/a1f7acf65795451d89f0a38565a975b3_5.csv')
print(len(bike_locations))
bike_locations.head()

596


,OBJECTID,ID,ADDRESS,TERMINAL_NUMBER,LATITUDE,LONGITUDE,INSTALLED,LOCKED,INSTALL_DATE,REMOVAL_DATE,TEMPORARY_INSTALL,NUMBER_OF_BIKES,NUMBER_OF_EMPTY_DOCKS,X,Y,SE_ANNO_CAD_DATA,OWNER
0,328739114,347,Connecticut Ave & McKinley St NW,31315,38.964544,-77.075135,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,3,16,393488.397567,144072.925481,NaN,DC
1,328739115,348,15th & L St NW,31276,38.903660,-77.034846,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,16,3,396977.478449,137312.167823,NaN,DC
2,328739116,349,17th & G St NW,31277,38.898300,-77.039732,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,25,4,396553.410741,136717.337924,NaN,DC
3,328739117,350,Spring St & 2nd Ave,32050,38.997653,-77.034499,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,0,15,397011.520184,147746.232006,NaN,NaN
4,328739118,351,18th & R St NW,31278,38.912648,-77.041834,YES,NO,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,NO,4,14,396371.800704,138310.163802,NaN,DC


In [21]:
# the data process reveals that there have been a few stations that have been removed or relocated. Out of the 667
# bikes in the system from 2010 to 2019 we have information of location for one hundred stations
list_stations_dataframe = pd.DataFrame(list_stations, columns = ['ADDRESS'])
list_stations_dataframe['dummy']= 1
list_stations_dataframe_2019 = list_stations_dataframe.merge(bike_locations, how='left', on='ADDRESS')
len(list_stations_dataframe_2019[list_stations_dataframe_2019['ID'].isnull()])

100

In [22]:
bike_trips_data_graph_2019 = bike_trips_data_graph_2019.merge(bike_locations[['ADDRESS','ID']], 
                                                              how='left',
                                                              left_on='Start station',
                                                             right_on='ADDRESS')

bike_trips_data_graph_2019 = bike_trips_data_graph_2019.merge(bike_locations[['ADDRESS','ID']], 
                                                              how='left',
                                                              left_on='End station',
                                                             right_on='ADDRESS')

bike_trips_data_graph_2019.head()

,Start station,End station,trips,ADDRESS_x,ID_x,ADDRESS_y,ID_y
0,10th & E St NW,10th & Florida Ave NW,100,10th & E St NW,199.0,10th & Florida Ave NW,274.0
1,10th & E St NW,10th & G St NW,419,10th & E St NW,199.0,10th & G St NW,334.0
2,10th & E St NW,10th & K St NW,1165,10th & E St NW,199.0,10th & K St NW,159.0
3,10th & E St NW,10th & Monroe St NE,63,10th & E St NW,199.0,10th & Monroe St NE,93.0
4,10th & E St NW,10th & U St NW,319,10th & E St NW,199.0,10th & U St NW,43.0


In [23]:
graph_data_2019 = bike_trips_data_graph_2019[['Start station', 'End station','trips']]
graph_data_2019 = graph_data_2019.rename(columns = {'Start station':'source', 
                                                    'End station':'target',
                                                    'trips':'weight'})
graph_data_2019.to_csv('../data/capital_bikes/graph_data_2019.csv', index=False)

In [24]:
# save data for 2020
bike_trips_data_graph_2020.to_csv('../data/capital_bikes/graph_data_2020.csv', index=False)

In [26]:
# merge the data for 2019 and 2020
full_data = pd.concat([bike_trips_data_graph_2020,graph_data_2019])
print(len(full_data))
full_data = full_data.groupby(['source', 'target']).sum('weight').reset_index()
print(len(full_data))
full_data.head()

223830
150527


,source,target,weight
0,10th & E St NW,10th & Florida Ave NW,108
1,10th & E St NW,10th & G St NW,432
2,10th & E St NW,10th & K St NW,1192
3,10th & E St NW,10th & Monroe St NE,79
4,10th & E St NW,10th & U St NW,330


In [27]:
# save data from bike locations
bike_locations.to_csv('../data/capital_bikes/bike_locations.csv', index=False)

In [28]:
# save the full data
full_data.to_csv('../data/capital_bikes/graph_data_full.csv', index=False)